In [3]:
import pandas as pd
import io
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink
from pathlib import Path
import pyexcel as pe
import base64
import zipfile

# Étape 1 : Création du widget d'upload
upload = widgets.FileUpload(accept='.xls', multiple=True)
display(upload)

# Étape 2 : Fonction de nettoyage
def clean_column_name(name):
    return "qte" if str(name).strip().lower() == "quantité" else name

# Étape 3 : Conversion et stockage des fichiers
converted_files = []

def on_upload_change(change):
    if upload.value:
        converted_files.clear()  # Nettoyer la liste à chaque nouvelle importation
        for filedata in upload.value:
            filename = filedata['name']
            content = io.BytesIO(filedata['content'])

            try:
                book = pe.get_book(file_type="xls", file_stream=content)
                sheet = book[0]

                data = list(sheet.rows())
                if not data:
                    raise ValueError("Feuille vide")

                headers = data[0]
                rows = data[1:]
                df = pd.DataFrame(rows, columns=headers)

                # Nettoyage des colonnes
                df.columns = [clean_column_name(col) for col in df.columns]

                # Conversion float → int si applicable
                for col in df.columns:
                    df[col] = df[col].apply(lambda x: int(x) if isinstance(x, float) and x.is_integer() else x)

                # Enregistrement en fichier CSV (sur disque pour le zippage)
                output_filename = f"{Path(filename).stem}.csv"
                df.to_csv(output_filename, index=False, sep=';', encoding='cp1252', lineterminator='\r\n')
                converted_files.append(output_filename)

                print(f"✅ Fichier converti : {output_filename}")

            except Exception as e:
                print(f"❌ Erreur pour {filename} : {e}")

        # Une fois toutes les conversions faites, créer le zip
        if converted_files:
            zip_name = "fichiers_convertis.zip"
            with zipfile.ZipFile(zip_name, 'w') as zipf:
                for file in converted_files:
                    zipf.write(file, arcname=file)
            print(f"\n📦 Archive créée : {zip_name}")
            display(FileLink(zip_name))

# Étape 4 : Attacher l'événement
upload.observe(on_upload_change, names='value')


FileUpload(value=(), accept='.xls', description='Upload', multiple=True)